<a href="https://colab.research.google.com/github/younguk072023/Pytorch_study/blob/main/ResNet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!ls /content/drive/MyDrive/'딥러닝 논문_코드구현'/pcamv1




In [ ]:
import gzip
import shutil
from pathlib import Path

src_dir = Path("/content/drive/MyDrive/딥러닝 논문_코드구현/pcamv1")
dst_dir = Path("pcam")

dst_dir.mkdir(exist_ok=True)

for src in src_dir.iterdir():
  if src.suffix == ".gz":
    with gzip.open(src, "rb") as f_in:
      dst=dst_dir / src.stem
      with open(dst, "wb")as f_out:
        shutil.copyfileobj(f_in,f_out)

  else:
    dst=dst_dir/src.name
    shutil.copy2(src,dst)

In [ ]:
import torch
from torchvision.transforms import v2

trn_transforms = v2.Compose([
    v2.ToImage(),
    v2.RandomResizedCrop(size=(96,96),antialias=True),
    v2.RandomHorizontalFlip(p=0.5),#이미지 50%확률로 수평 반전
    v2.RandomVerticalFlip(p=0.5),#이미지가 50% 확률로 수직 반전
    v2.ToDtype(torch.float32, scale=True),#픽셀값 0에서1 범위로 정규
    v2.Normalize(mean=[0.485,0.456,0.406], std=[0.229,0.224,0.225]),#이미지넷 정규화 기준


])

test_transforms = v2.Compose([
    v2.ToImage(),
    v2.ToDtype(torch.float32,scale=True),
    v2.Normalize(mean=[0.485,0.456,0.406],std=[0.229,0.224,0.225])


])

In [ ]:
from torchvision.datasets import PCAM
trn_dataset = PCAM(".", split="train",transform=trn_transforms)
test_dataset = PCAM(".",split="test", transform=test_transforms)

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

def imshow(inp):
  """Display image for Tensor."""
  inp=inp.numpy().transpose((1,2,0))#채널,높이,너비는 기본적인 파이토치, matplot에서는 높이, 너비, 채널 순
  mean=np.array([0.485,0.456,0.406])
  std=np.array([0.229,0.224,0.225])
  inp=np.clip(inp,0,1)
  plt.imshow(inp)
  plt.axis("off")
  plt.show()

In [ ]:
# 5개의 이미지를 가로로 표시
fig, axes = plt.subplots(1, 5, figsize=(15, 3))  # 1행 5열로 subplot 생성

for i in range(5):
    axes[i].imshow(trn_dataset[i][0].numpy().transpose((1, 2, 0)))
    axes[i].axis('off')  # 축 없애기

plt.show()


In [ ]:
import torch

trn_loader=torch.utils.data.DataLoader(trn_dataset,batch_size=64,shuffle=True, num_workers=2)
tst_loader=torch.utils.data.DataLoader(test_dataset,batch_size=64,shuffle=False,num_workers=2)


In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)


In [ ]:
from tqdm import tqdm

def train(model, criterion, optimizer, trn_loader, test_loader, num_epochs=25):
  for epoch in range(num_epochs):

    model.train()
    trn_loss=0.0
    trn_corrects=0
    for inputs, labels in tqdm(trn_loader):
      inputs=inputs.to(device)
      labels=labels.to(device)

      outputs = model(inputs)
      _, preds = torch.max(outputs,1)
      loss=criterion(outputs, labels)

      optimizer.zero_grad()
      loss.backward()
      optimizer.step()

      trn_loss+= loss.titem()*inputs.size(0)
      trn_corrects+=torch.sum(preds==labels.data)

    trn_epoch_loss= trn_loss/len(trn_loader.dataset)
    trn_epoch_acc=trn_corrects.double()/len(trn_loader.dataset)
    print(f"[Train] Loss : {trn_epoch_loss:.4f} Acc:{trn_epoch_acc:.4f}")

    with torch.no.grad():
      model.eval()
      test_loss=0.0
      test_corrects=0
      for inputs,labels in tqdm(test_loader):
        inputs=inputs.to(device)
        labels=labels.to(device)

        outputs=model(inputs)
        _,preds=torch.max(outputs,1)
        loss=criterion(outputs,labels)

        test_loss+=loss.item()*inputs.size(0)
        test_corrects+=torch.sum(preds==labels.data)

      test_epoch_loss=test_loss / len(test_loader.dataset)
      test_epoch_acc = test_corrects.double()/ len(test_loader.dataset)
      print(f"[Test] Loss: {test_epoch_loss:.4f} Acc: {test_epoch_acc:,4f}")



In [ ]:
import torch.nn as nn
import torch.optim as optim
from torchvision.models import resnet18, ResNet18_Weights

model = resnet18(weights=ResNet18_Weights.DEFAULT)

model=model.to(device)

In [ ]:
num_ftrs=model.fc.in_features
model.fc=nn.Linear(num_ftrs,2)

In [ ]:
criterion=nn.CrossEntropyLoss()
optimizer=optim.Adam(model.parameters(),lr=0.001)

In [ ]:
train(model, criterion, optimizer, trn_loader, tst_loader, num_epochs=10)

In [ ]:
torch.save(model.state_dict(), "/cotent/drive/MyDrive/model_pcam.pth")
model.load_state_dict(torch.load("/content/drive/MyDrive/model_pcam.pth"))

In [ ]:
import matplotlib.pyplot as plt

def visualize_model(model, num_images=6):
  model.eval()
  images_so_far=0

  with torch.no_grad():
    for i, (inouts,labels) in enumerate(tst_loader):
      inputs= inputs.to(device)
      labels=labels.to(device)

      outputs=model(inputs)
      _,preds = torch.max(outputs,1)

      for j in range(inputs.size()[0]):
        images_so_far+=1
        imshow(inputs.cpu()[j])
        print(f'predicted:{preds[j]}')
        print(f"label:{labels[j]}")